In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
!pip install jetnet
from jetnet.datasets import JetNet
from jetnet.datasets.normalisations import FeaturewiseLinear
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.1/865.1 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.7/636.7 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.8/700.8 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.2/466.2 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.3/362.3 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [3]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Function to one-hot encode the jet type and leave the rest of the features as is
def OneHotEncodeType(x: np.ndarray):
    enc = OneHotEncoder(categories=[[0., 2., 3.]])
    type_encoded = enc.fit_transform(x[..., 0].reshape(-1, 1)).toarray()
    other_features = x[..., 1:].reshape(-1, 3)
    return np.concatenate((type_encoded, other_features), axis=-1).reshape(*x.shape[:-1], -1)


In [5]:
data_args = {
    "jet_type": ["g", "t", "w"],
    "data_dir": "./datasets/jetnet",
    "particle_features": ["etarel", "phirel", "ptrel", "mask"],
    "jet_features": ["type", "pt", "eta", "mass"],
    "particle_normalisation": FeaturewiseLinear(
        normal=True, normalise_features=[True, True, True, False]
    ),
    "jet_transform": OneHotEncodeType,
    "download": True,
}

jets_train = JetNet(**data_args, split="train")
jets_valid = JetNet(**data_args, split="valid")

x_train = jets_train.particle_data
y_train = jets_train.jet_data[:, 0]
x_valid = jets_valid.particle_data
y_valid = jets_valid.jet_data[:, 0]

# Convert to PyTorch tensors and move to GPU
x_train = torch.from_numpy(x_train).to(device)
x_valid = torch.from_numpy(x_valid).to(device)

y_train = y_train.reshape(-1, 1)
y_valid = y_valid.reshape(-1, 1)

[██████████████████████████████████████████████████] 100%
[██████████████████████████████████████████████████] 100%
[██████████████████████████████████████████████████] 100%


In [6]:
enc = OneHotEncoder(sparse_output=False)
y_train_encoded = enc.fit_transform(y_train)
y_valid_encoded = enc.fit_transform(y_valid)

y_train_indices = torch.from_numpy(np.argmax(y_train_encoded, axis=1)).to(device)
y_valid_indices = torch.from_numpy(np.argmax(y_valid_encoded, axis=1)).to(device)

In [7]:
# Define DeepSet Model
class DeepSet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(DeepSet, self).__init__()
        self.phi = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        self.rho = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        phi_output = self.phi(x)  # Shape: [n, c, hidden_dim]
        aggregated = torch.sum(phi_output, dim=1)  # Shape: [n, hidden_dim]
        output = self.rho(aggregated)  # Shape: [n, output_dim]
        return output


In [8]:
# Model Parameters
input_dim = 4
hidden_dim = 64
output_dim = 3

In [11]:
# Initialize model, loss, and optimizer, and move model to GPU
model = DeepSet(input_dim, hidden_dim, output_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 500
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(x_train)
    loss = criterion(outputs, y_train_indices)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/500], Loss: 1.0832
Epoch [20/500], Loss: 1.0630
Epoch [30/500], Loss: 1.0446
Epoch [40/500], Loss: 1.0223
Epoch [50/500], Loss: 0.9943
Epoch [60/500], Loss: 0.9529
Epoch [70/500], Loss: 0.8991
Epoch [80/500], Loss: 0.8469
Epoch [90/500], Loss: 0.8179
Epoch [100/500], Loss: 0.7996
Epoch [110/500], Loss: 0.7895
Epoch [120/500], Loss: 0.7795
Epoch [130/500], Loss: 0.7689
Epoch [140/500], Loss: 0.7560
Epoch [150/500], Loss: 0.7446
Epoch [160/500], Loss: 0.7262
Epoch [170/500], Loss: 0.7088
Epoch [180/500], Loss: 0.6941
Epoch [190/500], Loss: 0.6701
Epoch [200/500], Loss: 0.6550
Epoch [210/500], Loss: 0.6385
Epoch [220/500], Loss: 0.6287
Epoch [230/500], Loss: 0.6190
Epoch [240/500], Loss: 0.6030
Epoch [250/500], Loss: 0.5954
Epoch [260/500], Loss: 0.5848
Epoch [270/500], Loss: 0.5792
Epoch [280/500], Loss: 0.5731
Epoch [290/500], Loss: 0.5697
Epoch [300/500], Loss: 0.5691
Epoch [310/500], Loss: 0.5665
Epoch [320/500], Loss: 0.5568
Epoch [330/500], Loss: 0.5565
Epoch [340/500], Lo

In [12]:
# Predict
with torch.no_grad():
    predictions = torch.argmax(model(x_valid), dim=1)

# Move predictions back to CPU for evaluation
accuracy = accuracy_score(y_valid_indices.cpu(), predictions.cpu())
print(f"Validation Accuracy: {accuracy}")

Validation Accuracy: 0.8011520881597897
